<a href="https://colab.research.google.com/github/mtermor/NTIC_DeepLearning/blob/main/NLP/02_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/eduardofc/data/main/amazon_sports.csv")
df['review_body'] = df['review_body'].str.replace('[^a-zA-ZñÑáéíóú .,:;]', '', regex=True)
df['review_body'] = df['review_body'].str.lower()
df.head()

,stars,review_body,review_title,product_category
0,1,nunca llego el pedido y el vendedor pasa de todo no contestan,No llego nunca,sports
1,1,"no sé como es, porque debería haber llegado ayer día de marzo, y hoy por la noche sigo esperando que llegue el frontal. y me habéis mandado el formulario de opinión. pues de momento mala no ha cumplido con la fecha de entrega.",Todavía no ha llegado,sports
2,1,"guantes cómodos, no lo niego, pero de mala calidad. yo creo que en caso de caída no valdrian para mucho, dos meses de uso y se están rajando.",Guantes de baja calidad,sports
3,1,hasta hoy no he visto el producto. el pedido hace ya casi mes. y notifico que he usado prime para está compra.,Muy Mala experiencia,sports
4,1,"no puedo valorarla porque, después de casi una semana, aún no he recibido mi pedido. pienso que amazon tendría que valorar las compañías de transporte con que trabaja, porque es indignante que pague mi cuota prime y nunca reciba mi pedido el día que toca",Paquete perdido?,sports


In [3]:
df.groupby('stars').size()

stars
1    2438
2    2551
3    2828
4    2860
5    2512
dtype: int64

In [4]:
""" Clasificamos en positivo (stars 4-5) y negativo (stars 1-2) """
df = df[df.stars != 3]
df['bad_product'] = (df.stars < 3).astype(int)
df.groupby('bad_product').size()

bad_product
0    5372
1    4989
dtype: int64

# Modelling for Classification

## Inputs (preprocessing)

In [5]:
X = df.review_body.values
y = df.bad_product

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer

vocab_size = 10000
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X)
# tokenizer.index_word
X_tokenized = tokenizer.texts_to_sequences(X)
# X_tokenized

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 50
X_padded = pad_sequences(X_tokenized, maxlen=max_len, truncating='post')
# X_padded

## Model (NN)

In [8]:
import tensorflow.keras as keras

keras.utils.set_random_seed(812)

In [9]:
from keras import Sequential
from keras.layers import Flatten, Dense, Embedding

embedding_dim = 2

model = keras.Sequential()

# Capas de embeddings
model.add(Embedding(input_length= max_len, input_dim=vocab_size, output_dim=embedding_dim))
model.add(Flatten())
# Capas de clasificación
# model.add(Dense(6, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [10]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 2)             20000     
                                                                 
 flatten (Flatten)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 20101 (78.52 KB)
Trainable params: 20101 (78.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
n_epochs = 20

model.fit(X_padded, y, epochs= n_epochs)

Epoch 1/20
324/324 [==============================] - 22s 60ms/step - loss: 0.6621 - accuracy: 0.6195
Epoch 2/20
324/324 [==============================] - 4s 11ms/step - loss: 0.5053 - accuracy: 0.7949
Epoch 3/20
324/324 [==============================] - 3s 10ms/step - loss: 0.3877 - accuracy: 0.8505
Epoch 4/20
324/324 [==============================] - 3s 9ms/step - loss: 0.3293 - accuracy: 0.8751
Epoch 5/20
324/324 [==============================] - 3s 9ms/step - loss: 0.2912 - accuracy: 0.8906
Epoch 6/20
324/324 [==============================] - 2s 5ms/step - loss: 0.2626 - accuracy: 0.9024
Epoch 7/20
324/324 [==============================] - 2s 5ms/step - loss: 0.2387 - accuracy: 0.9147
Epoch 8/20
324/324 [==============================] - 2s 6ms/step - loss: 0.2188 - accuracy: 0.9239
Epoch 9/20
324/324 [==============================] - 2s 5ms/step - loss: 0.2021 - accuracy: 0.9296
Epoch 10/20
324/324 [==============================] - 2s 7ms/step - loss: 0.1874 - accuracy: 0.

# Embeddings Layer

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 2)             20000     
                                                                 
 flatten (Flatten)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 20101 (78.52 KB)
Trainable params: 20101 (78.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
embed_layer = model.layers[0]
embed_weights = embed_layer.get_weights()[0]
print(embed_weights.shape)
embed_weights

(10000, 2)


array([[-0.05058567, -0.04176808],
       [-0.1910364 ,  0.2953519 ],
       [ 0.06250687, -0.03166124],
       ...,
       [ 0.09126089, -0.05233277],
       [ 0.29651302, -0.2588437 ],
       [ 0.22588553, -0.25242013]], dtype=float32)

In [23]:
for v in ['bueno', 'malo', 'horrible']:
  k = tokenizer.word_index[v]
  print(f'{v}: {k}: {embed_weights[k]}')

bueno: 92: [-0.37987083  0.4268919 ]
malo: 215: [-0.05720437 -0.50714517]
horrible: 1265: [ 0.7407763 -0.7420258]


In [24]:
words = []
x_axis = []
y_axis = []

for k, v in tokenizer.index_word.items():
  x, y = embed_weights[k]
  words.append(v)
  x_axis.append(x)
  y_axis.append(y)
  if k == 1500:
    break

In [26]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

output_notebook()

source = ColumnDataSource(data=dict(x=x_axis, y=y_axis, words=words))
p = figure(tools='', title='2-dim embeddings space')
p.circle('x', 'y', size=2, source=source)
hover = HoverTool()
hover.tooltips = [('Palabra', '@words'), ('(x,y)','($x, $y)')]
p.add_tools(hover)
show(p)

# Distancia del coseno
Similitud del coseno

In [27]:
for v in ['bueno', 'malo', 'horrible']:
  k = tokenizer.word_index[v]
  print(f'{v}: {k}: {embed_weights[k]}')

bueno: 92: [-0.37987083  0.4268919 ]
malo: 215: [-0.05720437 -0.50714517]
horrible: 1265: [ 0.7407763 -0.7420258]


In [28]:
import numpy as np

# Recomendacion de películas
# (comedia, intriga, terror, thriller, aventuras, sci-fi) n=6 dimensiones

v_toy_story     = np.array([3, 1, 0, 1, 5, 4])
v_mission_imp   = np.array([2, 5, 0, 5, 4, 2])
v_jurassic_park = np.array([1, 2, 1, 3, 5, 3])
v_exorcista     = np.array([0, 4, 5, 3, 1, 2])

In [30]:
print(np.dot(v_toy_story, v_mission_imp))
print(np.dot(v_toy_story, v_jurassic_park))
print(np.dot(v_toy_story, v_exorcista))

44
45
20


In [31]:
# Una película con todo 5s tendría muchos más puntos independientemente de con que
# El que tenga más "puntos" en el vector es el que más puntos tiene en el producto escalar

In [32]:
print(v_toy_story.sum())
print(v_mission_imp.sum())
print(v_jurassic_park.sum())
print(v_exorcista.sum())


14
18
15
15


In [35]:
print(np.linalg.norm(v_toy_story))
print(np.linalg.norm(v_mission_imp))
print(np.linalg.norm(v_jurassic_park))
print(np.linalg.norm(v_exorcista))

7.211102550927978
8.602325267042627
7.0
7.416198487095663


In [37]:
print(np.dot(v_toy_story/ np.linalg.norm(v_toy_story), v_mission_imp / np.linalg.norm(v_mission_imp)))

0.7093084682410772


In [40]:
print(np.dot(v_toy_story, v_mission_imp) / (np.linalg.norm(v_toy_story) * np.linalg.norm(v_mission_imp)))
print(np.dot(v_toy_story, v_jurassic_park) / (np.linalg.norm(v_toy_story) * np.linalg.norm(v_jurassic_park)))
print(np.dot(v_toy_story, v_exorcista) / (np.linalg.norm(v_toy_story) * np.linalg.norm(v_exorcista)))

0.7093084682410772
0.8914824582191182
0.3739787960033829


In [42]:
def dist_cosine(v1, v2):
  d = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
  return d

# dist_cosine(v_toy_story, v_exorcista)

0.3739787960033829

In [47]:
for v in ['bueno', 'malo', 'horrible', 'encantado','mucho']:
  k = tokenizer.word_index[v]
  print(f'{v}: {k}: {embed_weights[k]}')

bueno: 92: [-0.37987083  0.4268919 ]
malo: 215: [-0.05720437 -0.50714517]
horrible: 1265: [ 0.7407763 -0.7420258]
encantado: 285: [-1.0762715  1.2655139]
mucho: 41: [-0.09403238  0.07717179]


In [49]:
v_bueno = embed_weights[92]
v_malo = embed_weights[215]
v_horrible = embed_weights[1265]
v_encantado = embed_weights[285]
v_mucho = embed_weights[41]


print(f'bueno y malo: {dist_cosine(v_bueno, v_malo):.3f}')
print(f'bueno y horrible: {dist_cosine(v_bueno, v_horrible):.3f}')
print(f'horrible y malo: {dist_cosine(v_horrible, v_malo):.3f}')
print(f'horrible y encantado: {dist_cosine(v_horrible, v_encantado):.3f}')
print(f'bueno y encantado: {dist_cosine(v_bueno, v_encantado):.3f}')
print(f'bueno y mucho: {dist_cosine(v_bueno, v_mucho):.3f}')

bueno y malo: -0.668
bueno y horrible: -0.998
horrible y malo: 0.624
horrible y encantado: -0.997
bueno y encantado: 1.000
bueno y mucho: 0.988
